In [1]:
!kaggle competitions download -c spaceship-titanic

spaceship-titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# First I am going for a very simple model with dropping all the nan values and converting the categorical objects into dummy

In [4]:
from torch.utils.data import Dataset
class tit_space(Dataset):
    def __init__(self, to_ = 'train'):
        df = pd.read_csv(f'spaceship-titanic/{to_}.csv')

        df.dropna(inplace = True)
        df.drop(['Name', 'PassengerId', 'Cabin'], axis = 1, inplace = True)
        df = pd.get_dummies(df, columns = ['HomePlanet', 'Destination'], drop_first = True)
        df.replace([False, True], [0,1], inplace = True)
        df = (df-df.min())/(df.max() - df.min())
        # print(df)
        self.label = torch.tensor(df.pop('Transported').to_numpy(), dtype = torch.float)
        self.df = torch.tensor(df.to_numpy(), dtype = torch.float)
    
    def __getitem__(self, idx):
        return self.df[idx], self.label[idx]
    
    def __len__(self):
        return self.df.shape[0]


In [5]:
torch.manual_seed(42)
from torch.utils.data import DataLoader
from torch.utils.data import random_split

tit_data = tit_space()
train_dataset, test_dataset = random_split(tit_data, [0.9, 0.1])
len(train_dataset), len(test_dataset)
tit_data[0]

(tensor([0.0000, 0.4937, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 1.0000]),
 tensor(0.))

In [6]:
# Hyperparameter----

batch_size = 32
lr = 1e-4
epoch = 11
batch_eval_inter = 100
eval_train_test = 10
# eval_test = 10

# ------------------

In [7]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [8]:
# Model
torch.manual_seed(42)
class space_finder(nn.Module):
    def __init__(self):
        super(space_finder, self).__init__() 
        self.l1 = nn.Sequential(
                nn.Linear(12, 100),
                nn.BatchNorm1d(100),
                nn.ReLU())
        
        self.l2 = nn.Sequential(
                nn.Linear(100, 200),
                nn.BatchNorm1d(200),
                nn.ReLU())
        self.l3 = nn.Sequential(
                nn.Linear(200, 500),
                nn.BatchNorm1d(500),
                nn.ReLU())
        self.l4 = nn.Sequential(
                nn.Linear(500, 200),
                nn.BatchNorm1d(200),
                nn.ReLU())
        self.l5 = nn.Sequential(
                nn.Linear(200, 100),
                nn.BatchNorm1d(100),
                nn.ReLU())
        self.logits = nn.Linear(100, 1)

    def forward(self, x):
        
        x1 = self.l1(x)
        x2 = self.l2(x1)
        x = self.l3(x2)
        x = x2 + self.l4(x)
        x = x1 + self.l5(x)
        logits = self.logits(x)
        return logits
        

In [9]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'cpu'

In [139]:
model = space_finder()
model.to(device)

TypeError: space_finder.__init__() missing 3 required positional arguments: 'n_embd', 'n_heads', and 'n_layers'

In [307]:
#experimentation
# torch.manual_seed(42)
# x = torch.rand((1,5,12), device = device)
# y = model(x)
# y

In [308]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [11]:
import time
from sklearn.metrics import accuracy_score

def train_loop(i, see_batch_loss = False):
    model.train()
    total_loss = 0
    y_true = []
    y_preds = []
    for batch, (data, label) in enumerate(train_loader):
        data , label = data.to(device), label.to(device)
        # print(data.shape)
        logits = model(data)
        # preds = preds.argmax(dim = -1)
        # print(logits)
        optimizer.zero_grad()
        loss = loss_fn(logits.view(-1), label)
        loss.backward()
        optimizer.step()
        sche
        
        total_loss += loss.item()
        preds = torch.sigmoid(logits).view(-1) > 0.5
        y_true.extend(label.cpu().tolist())
        y_preds.extend(preds.detach().cpu().tolist())
        
        if see_batch_loss:
            if batch%eval_train == 0:
                print(f'Batch_Loss_{batch} : {loss.item()}')

    if i%eval_train_test==0:
        val_loss, val_acc = test_loop(test_loader)
        print(f'Epoch {i+1}: train_loss: {(total_loss/len(train_loader)):.4f}, train_acc: {(accuracy_score(y_true, y_preds)):.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}')
        


def test_loop(dataset):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        y_true = []
        y_preds = []
        for data, label in dataset:
            data , label = data.to(device), label.to(device)
            logits = model(data)
    
            loss = loss_fn(logits.view(-1), label)
            
            total_loss+=loss.item()
            preds = torch.sigmoid(logits).view(-1) > 0.5
            y_true.extend(label.cpu().tolist())
            y_preds.extend(preds.detach().cpu().tolist())
                              
    return total_loss/len(test_loader), accuracy_score(y_true, y_preds)
    # print(f'val_loss: {total_loss/len(test_loader)}, val_acc: {accuracy_score(y_true, y_preds)}')  

In [315]:
start = time.time()

for i in range(epoch): 
    train_loop(i)
    # break
    
end = time.time()


print(f'Total_time: {end-start}')

Epoch 1: train_loss: 0.3913, train_acc: 0.8105, val_loss: 0.4382, val_acc: 0.8061
Total_time: 24.17899179458618


In [ ]:
# I trained the model on the 5 layer mlp and the result was not that great i got stuck at 80% and I even trained it for 1000 epochs!
# Now I will try to use skip connections, lets see..


#### Lets try a transformer model here!!!!
1. First layer will be layer norm
2. Second will be attention layer
   1. First I will try for a single head attention
   2. Multihead attention
   3. multiblocks


In [131]:
# Hyperparameter----

batch_size = 256
lr = 1e-2
epoch = 100
batch_eval_inter = 100
eval_train_test = 10
n_embd = 512
n_heads = 8
n_layers = 6
# eval_test = 10

# ------------------

In [132]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [133]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)

In [134]:
import time
from sklearn.metrics import accuracy_score

def train_loop(i, see_batch_loss = False):
    model.train()
    total_loss = 0
    y_true = []
    y_preds = []
    for batch, (data, label) in enumerate(train_loader):
        data , label = data.to(device), label.to(device)
        # print(data.shape)
        logits = model(data)
        # preds = preds.argmax(dim = -1)
        # print(logits)
        optimizer.zero_grad()
        loss = loss_fn(logits.view(-1), label)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        preds = torch.sigmoid(logits).view(-1) > 0.5
        y_true.extend(label.cpu().tolist())
        y_preds.extend(preds.detach().cpu().tolist())
        
        if see_batch_loss:
            if batch%eval_train == 0:
                print(f'Batch_Loss_{batch} : {loss.item()}')

    if i%eval_train_test==0:
        val_loss, val_acc = test_loop(test_loader)
        print(f'Epoch {i+1}: train_loss: {(total_loss/len(train_loader)):.4f}, train_acc: {(accuracy_score(y_true, y_preds)):.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}')
        


def test_loop(dataset):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        y_true = []
        y_preds = []
        for data, label in dataset:
            data , label = data.to(device), label.to(device)
            logits = model(data)
    
            loss = loss_fn(logits.view(-1), label)
            
            total_loss+=loss.item()
            preds = torch.sigmoid(logits).view(-1) > 0.5
            y_true.extend(label.cpu().tolist())
            y_preds.extend(preds.detach().cpu().tolist())
                              
    return total_loss/len(test_loader), accuracy_score(y_true, y_preds)
    # print(f'val_loss: {total_loss/len(test_loader)}, val_acc: {accuracy_score(y_true, y_preds)}')  

In [135]:
# Single Head
from torch.nn import functional as F
class sa_layer(nn.Module):
    def __init__(self, head_size):
        
        super(sa_layer, self).__init__()
        self.q = nn.Linear(n_embd, head_size, bias = False)
        self.k = nn.Linear(n_embd, head_size, bias = False)
        self.v = nn.Linear(n_embd, head_size, bias = False)

    def forward(self, x):

        q = self.q(x)
        k = self.k(x)

        wei = q @ k.transpose(0,1) * 100  ** -0.5
        wei = F.softmax(wei, dim = -1)

        v = self.v(x)

        out = wei @ v

        return out

class multihead_attention(nn.Module):
    def __init__(self, n_embd, head_size):
        
        super(multihead_attention, self).__init__()
        self.heads = nn.ModuleList([sa_layer(head_size) for _ in range(n_heads)])
        self.ll = nn.Linear(n_embd, n_embd)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], axis = -1)
        out = self.ll(out)
        
        return out

class blocks(nn.Module):
    def __init__(self, n_embd, n_heads):
        super(blocks, self).__init__()
    
        head_size = n_embd // n_heads
        self.ln1 = nn.LayerNorm(n_embd)        
        self.multihead = nn.Sequential(*[multihead_attention(n_embd, head_size) for _ in range(n_layers)])
        
        self.ln2 = nn.LayerNorm(n_embd)
        self.ffw = nn.Sequential(
                    nn.Linear(n_embd, n_embd * 4),
                    nn.ReLU(),
                    nn.Linear(n_embd * 4, n_embd),
        )
    def forward(self, x):
        multihead = x + self.multihead(self.ln1(x))
        return multihead + self.ffw(self.ln2(multihead))

In [136]:
# Model
torch.manual_seed(42)
class space_finder(nn.Module):
    def __init__(self, n_embd, n_heads, n_layers):
        super(space_finder, self).__init__()
        self.l1 = nn.Linear(12, n_embd)
               
        self.blocks = blocks(n_embd , n_heads)

        self.logits = nn.Linear(n_embd, 1)

    def forward(self, x):
        
        x = self.l1(x)

        x = self.blocks(x)

        logits = self.logits(x)
        
        return logits
        

In [137]:
model = space_finder(n_embd, n_heads, n_layers)
model.to(device)

space_finder(
  (l1): Linear(in_features=12, out_features=512, bias=True)
  (blocks): blocks(
    (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (multihead): Sequential(
      (0): multihead_attention(
        (heads): ModuleList(
          (0-7): 8 x sa_layer(
            (q): Linear(in_features=512, out_features=64, bias=False)
            (k): Linear(in_features=512, out_features=64, bias=False)
            (v): Linear(in_features=512, out_features=64, bias=False)
          )
        )
        (ll): Linear(in_features=512, out_features=512, bias=True)
      )
      (1): multihead_attention(
        (heads): ModuleList(
          (0-7): 8 x sa_layer(
            (q): Linear(in_features=512, out_features=64, bias=False)
            (k): Linear(in_features=512, out_features=64, bias=False)
            (v): Linear(in_features=512, out_features=64, bias=False)
          )
        )
        (ll): Linear(in_features=512, out_features=512, bias=True)
      )
      (2): mul

In [138]:
start = time.time()

for epoch in range(epoch): 
    train_loop(epoch)
    # break
    
end = time.time()


print(f'Total_time: {end-start}')

Epoch 1: train_loss: 0.7199, train_acc: 0.3690, val_loss: 0.7200, val_acc: 0.3727
Epoch 11: train_loss: 0.7203, train_acc: 0.3687, val_loss: 0.7200, val_acc: 0.3727
Epoch 21: train_loss: 0.7198, train_acc: 0.3685, val_loss: 0.7200, val_acc: 0.3727
Epoch 31: train_loss: 0.7205, train_acc: 0.3690, val_loss: 0.7200, val_acc: 0.3727
Epoch 41: train_loss: 0.7210, train_acc: 0.3687, val_loss: 0.7200, val_acc: 0.3727
Epoch 51: train_loss: 0.7204, train_acc: 0.3687, val_loss: 0.7200, val_acc: 0.3727
Epoch 61: train_loss: 0.7208, train_acc: 0.3687, val_loss: 0.7200, val_acc: 0.3727
Epoch 71: train_loss: 0.7196, train_acc: 0.3690, val_loss: 0.7200, val_acc: 0.3727
Epoch 81: train_loss: 0.7201, train_acc: 0.3688, val_loss: 0.7200, val_acc: 0.3727
Epoch 91: train_loss: 0.7199, train_acc: 0.3690, val_loss: 0.7200, val_acc: 0.3727
Total_time: 178.02459144592285
